In [23]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

In [89]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [308]:
train_data = [
    ["hungry", "FOOD"],
    ["Do not eat anything", "FOOD"],
    ["Recommend restaurant", "FOOD"],
    ["Good food near restaurant", "FOOD"],
    ["food", "FOOD"],
    ["What are you eating?", "FOOD"],
    ["I want to eat rice", "FOOD"],
    ["I want to watch movies", "MEDIA"],
    ["Have you seen anything lately?", "MEDIA"],
    ["Movie or entertainment recommendation", "MEDIA"],
    ["Show me some funny drama", "MEDIA"],
    ["Give me a plot with love", "MEDIA"],
    ["Back to the horror comedy", "MEDIA"],
    ["Show only funny video highlights", "MEDIA"]
]
test_data = [
    ["Find a restaurant", "FOOD"],
    ["I want to eat spicy rice cake", "FOOD"],
    ["I want cake", "FOOD"],
    ["Kamal's nayagan movie", "MEDIA"]
]

In [267]:
#train_data=import_data()
#from random import shuffle
#shuffle(train_data)
train_X,train_Y = list(zip(*train_data))

In [268]:
train_X = tokenize_corpus(train_X)
#print train_X
print len(train_X)
#print train_Y
print len(train_Y)

14
14


In [269]:
word2index={'<unk>' : 0}
for x in train_X:
    for token in x:
        if word2index.get(token)==None:
            word2index[token]=len(word2index)
            
class2index = {'FOOD' : 0, 'MEDIA' : 1}
#class2index = {'NEG' : 0, 'POS' : 1}
#print(word2index)
#print(class2index)
print len(word2index)

45


In [270]:
def make_BoW(seq,word2index):
    tensor = torch.zeros(len(word2index))
    for w in seq:
        index = word2index.get(w)
        if index!=None:
            tensor[index]+=1.
        else:
            index = word2index['<unk>']
            tensor[index]+=1.
    
    return tensor

In [271]:
make_BoW(['Do', 'not', 'eat', 'anything'],word2index).view(1,-1)



Columns 0 to 12 
    0     0     1     1     1     1     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 39 to 44 
    0     0     0     0     0     0
[torch.FloatTensor of size 1x45]

In [272]:
train_X = torch.cat([Variable(make_BoW(x,word2index)).view(1,-1) for x in train_X])
train_y = torch.cat([Variable(torch.LongTensor([class2index[y]])) for y in train_Y])

In [273]:
#print train_Y
train_X.size()

torch.Size([14, 45])

In [274]:
class BoWClassifier(nn.Module):
    def __init__(self,vocab_size,output_size):
        super(BoWClassifier,self).__init__()
        
        self.linear = nn.Linear(vocab_size,output_size)
    
    def forward(self,inputs):
        return self.linear(inputs)

In [293]:
epochs = 100
LR = 0.2
model = BoWClassifier(len(word2index),2)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=LR)

In [294]:
for epoch in range(epochs):
    model.zero_grad()
    preds = model(train_X)
    loss = loss_function(preds,train_y)
    if epoch % 10 == 0:
        print 'epoch {} loss {} '.format(epoch,loss.data[0])
    loss.backward()
    optimizer.step()

epoch 0 loss 0.695884764194 
epoch 10 loss 0.476599872112 
epoch 20 loss 0.357526838779 
epoch 30 loss 0.284007668495 
epoch 40 loss 0.234425649047 
epoch 50 loss 0.198890700936 
epoch 60 loss 0.172271937132 
epoch 70 loss 0.151648715138 
epoch 80 loss 0.135239556432 
epoch 90 loss 0.121898189187 


In [291]:
index2class = {v:k for k,v in class2index.items()}
print index2class

{0: 'FOOD', 1: 'MEDIA'}


In [309]:
for test in test_data:
    X = (test[0])
    X = Variable(make_BoW(X,word2index)).view(1,-1)
    pred = model(X)
    pred = pred.max(1)[1].data[0]
    print("Input : %s" % test[0])
    print("Prediction : %s" % index2class[pred])
    print("Truth : %s" % test[1])
    print("\n")

Input : Find a restaurant
Prediction : FOOD
Truth : FOOD


Input : I want to eat spicy rice cake
Prediction : FOOD
Truth : FOOD


Input : I want cake
Prediction : FOOD
Truth : FOOD


Input : Kamal's nayagan movie
Prediction : MEDIA
Truth : MEDIA




In [174]:
"""def import_data(path = None):
    filenames=['data/positive.txt','data/negative.txt']
    data=[]
    for files in filenames:
        t=open(files)
        if files=='data/positive.txt':
               tag="POS"
        else:
               tag="NEG"
        s=[[i[:-1]] for i in t]
        for i in s:
            #print i
            data+=[i+[tag]]
        #print data
    return data"""